In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [56]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [3]:
import os

openai_api_key = os.environ.get('OPENAI_API_KEY')

In [4]:
db = SQLDatabase.from_uri("sqlite:///./movies.db")

In [111]:
llm = OpenAI(temperature=0, verbose=True)

In [7]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

In [101]:
db_chain.run("List actors of Harry Potter")



> Entering new SQLDatabaseChain chain...
List actors of Harry Potter
SQLQuery:SELECT people.name 
FROM people 
INNER JOIN stars ON people.id = stars.person_id 
INNER JOIN movies ON stars.movie_id = movies.id 
WHERE movies.title = 'Harry Potter'
SQLResult: []
Answer:No actors found for Harry Potter.
> Finished chain.


'No actors found for Harry Potter.'

In [17]:
db_chain2 = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True, return_intermediate_steps=True)

In [45]:
result = db_chain2("Answer hypothetical question about this database")



> Entering new SQLDatabaseChain chain...
Answer hypothetical question about this database
SQLQuery:SELECT * FROM movies LIMIT 5;
SQLResult: [(15724, 'Dama de noche', 1993), (23331, 'Pesn o geroyakh', 1983), (31458, 'El huésped del sevillano', 1970), (35423, 'Kate & Leopold', 2001), (36606, 'Another Time, Another Place', 1983)]
Answer:The database contains information about 5 movies: Dama de noche (1993), Pesn o geroyakh (1983), El huésped del sevillano (1970), Kate & Leopold (2001), and Another Time, Another Place (1983).
> Finished chain.


In [46]:
result["intermediate_steps"]

[{'input': "Answer hypothetical question about this database\nSQLQuery:SELECT * FROM movies LIMIT 5;\nSQLResult: [(15724, 'Dama de noche', 1993), (23331, 'Pesn o geroyakh', 1983), (31458, 'El huésped del sevillano', 1970), (35423, 'Kate & Leopold', 2001), (36606, 'Another Time, Another Place', 1983)]\nAnswer:",
  'top_k': '5',
  'dialect': 'sqlite',
  'table_info': '\nCREATE TABLE movies (\n\tid INTEGER, \n\ttitle TEXT NOT NULL, \n\tyear NUMERIC, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from movies table:\nid\ttitle\tyear\n15724\tDama de noche\t1993.0000000000\n23331\tPesn o geroyakh\t1983.0000000000\n31458\tEl huésped del sevillano\t1970.0000000000\n*/\n\n\nCREATE TABLE people (\n\tid INTEGER, \n\tname TEXT NOT NULL, \n\tbirth NUMERIC, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from people table:\nid\tname\tbirth\n1\tFred Astaire\t1899.0000000000\n2\tLauren Bacall\t1924.0000000000\n3\tBrigitte Bardot\t1934.0000000000\n*/\n\n\nCREATE TABLE directors (\n\tmovie_id INTEGER NOT NULL, \n\tperson_i

In [29]:
result["query"]

'How many movies have 5 stars?'

In [30]:
result["result"]

'At least 5 movies have 5 stars.'

In [35]:
isteps= result["intermediate_steps"]

In [39]:
isteps

[{'input': 'How many movies have 5 stars?\nSQLQuery:SELECT COUNT(*) FROM movies m INNER JOIN stars s ON m.id = s.movie_id GROUP BY m.id HAVING COUNT(*) = 5 LIMIT 5;\nSQLResult: [(5,), (5,), (5,), (5,), (5,)]\nAnswer:',
  'top_k': '5',
  'dialect': 'sqlite',
  'table_info': '\nCREATE TABLE movies (\n\tid INTEGER, \n\ttitle TEXT NOT NULL, \n\tyear NUMERIC, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from movies table:\nid\ttitle\tyear\n15724\tDama de noche\t1993.0000000000\n23331\tPesn o geroyakh\t1983.0000000000\n31458\tEl huésped del sevillano\t1970.0000000000\n*/\n\n\nCREATE TABLE people (\n\tid INTEGER, \n\tname TEXT NOT NULL, \n\tbirth NUMERIC, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from people table:\nid\tname\tbirth\n1\tFred Astaire\t1899.0000000000\n2\tLauren Bacall\t1924.0000000000\n3\tBrigitte Bardot\t1934.0000000000\n*/\n\n\nCREATE TABLE directors (\n\tmovie_id INTEGER NOT NULL, \n\tperson_id INTEGER NOT NULL, \n\tFOREIGN KEY(movie_id) REFERENCES movies (id), \n\tFOREIGN KEY(person_i

In [37]:
isteps[0]

{'input': 'How many movies have 5 stars?\nSQLQuery:SELECT COUNT(*) FROM movies m INNER JOIN stars s ON m.id = s.movie_id GROUP BY m.id HAVING COUNT(*) = 5 LIMIT 5;\nSQLResult: [(5,), (5,), (5,), (5,), (5,)]\nAnswer:',
 'top_k': '5',
 'dialect': 'sqlite',
 'table_info': '\nCREATE TABLE movies (\n\tid INTEGER, \n\ttitle TEXT NOT NULL, \n\tyear NUMERIC, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from movies table:\nid\ttitle\tyear\n15724\tDama de noche\t1993.0000000000\n23331\tPesn o geroyakh\t1983.0000000000\n31458\tEl huésped del sevillano\t1970.0000000000\n*/\n\n\nCREATE TABLE people (\n\tid INTEGER, \n\tname TEXT NOT NULL, \n\tbirth NUMERIC, \n\tPRIMARY KEY (id)\n)\n\n/*\n3 rows from people table:\nid\tname\tbirth\n1\tFred Astaire\t1899.0000000000\n2\tLauren Bacall\t1924.0000000000\n3\tBrigitte Bardot\t1934.0000000000\n*/\n\n\nCREATE TABLE directors (\n\tmovie_id INTEGER NOT NULL, \n\tperson_id INTEGER NOT NULL, \n\tFOREIGN KEY(movie_id) REFERENCES movies (id), \n\tFOREIGN KEY(person_id) R

In [38]:
isteps[1]

'SELECT COUNT(*) FROM movies m INNER JOIN stars s ON m.id = s.movie_id GROUP BY m.id HAVING COUNT(*) = 5 LIMIT 5;'

In [40]:
db_chain3 = SQLDatabaseChain.from_llm(llm, db, verbose=True, top_k=10, use_query_checker=True, return_intermediate_steps=True)

In [42]:
result3 = db_chain3("How many movies have 5 stars?")



> Entering new SQLDatabaseChain chain...
How many movies have 5 stars?
SQLQuery:SELECT COUNT(*) FROM movies m INNER JOIN stars s ON m.id = s.movie_id GROUP BY m.id HAVING COUNT(*) = 5 LIMIT 10;
SQLResult: [(5,), (5,), (5,), (5,), (5,), (5,), (5,), (5,), (5,), (5,)]
Answer:10 movies have 5 stars.
> Finished chain.


In [62]:
from langchain.chains import SQLDatabaseSequentialChain

In [131]:
chain4 = SQLDatabaseSequentialChain.from_llm(llm, db, verbose=True, top_k=20)

In [128]:
chain4.run("How many movies are released in 2020?")



> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['movies']

> Entering new SQLDatabaseChain chain...
How many movies are released in 2020?
SQLQuery:SELECT COUNT(*) FROM movies WHERE year = 2020;
SQLResult: [(3843,)]
Answer:3843 movies were released in 2020.
> Finished chain.

> Finished chain.


'3843 movies were released in 2020.'

In [132]:
chain4.run("List the titles and release years of all Harry Potter movies, in chronological order.")



> Entering new SQLDatabaseSequentialChain chain...
Table names to use:
['movies', 'people']

> Entering new SQLDatabaseChain chain...
List the titles and release years of all Harry Potter movies, in chronological order.
SQLQuery:SELECT title, year FROM movies WHERE title LIKE '%Harry Potter%' ORDER BY year ASC LIMIT 20;
SQLResult: [("Harry Potter and the Sorcerer's Stone", 2001), ('Harry Potter and the Chamber of Secrets', 2002), ('Harry Potter and the Prisoner of Azkaban', 2004), ('Harry Potter and the Goblet of Fire', 2005), ('Harry Potter and the Order of the Phoenix', 2007), ('Harry Potter and the Half-Blood Prince', 2009), ('Harry Potter and the Deathly Hallows: Part 1', 2010), ('Creating the World of Harry Potter, Part 4: Sound and Music', 2010), ('The Seekers Guide to Harry Potter', 2010), ('Harry Potter and the Deathly Hallows: Part 2', 2011), ('Harry Potter and the Untold Stories of Hogwarts', 2012), ('Harry Potter: A History of Magic', 2017), ('The Harry Potter Saga Analyze

'The Harry Potter movies, in chronological order, are "Harry Potter and the Sorcerer\'s Stone" (2001), "Harry Potter and the Chamber of Secrets" (2002), "Harry Potter and the Prisoner of Azkaban" (2004), "Harry Potter and the Goblet of Fire" (2005), "Harry Potter and the Order of the Phoenix" (2007), "Harry Potter and the Half-Blood Prince" (2009), "Harry Potter and the Deathly Hallows: Part 1" (2010), "Creating the World of Harry Potter, Part 4: Sound and Music" (2010), "The Seekers Guide to Harry Potter" (2010), "Harry Potter and the Deathly Hallows: Part 2" (2011), "Harry Potter and the Untold Stories of Hogwarts" (2012), and "Harry Potter: A History of Magic" (2017) and "The Harry Potter Saga Analyzed" (2018).'